In [3]:
#1 Set up

In [5]:
# 1.1 Install Dependencies
!pip install --upgrade pip setuptools wheel


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.8 MB 2.0 MB/s eta 0:00:01
   ------------ --------------------------- 0.6/1.8 MB 7.4 MB/s eta 0:00:01
   -------------------------------- ------- 1.5/1.8 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 40.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/67.1 kB ? eta -:--:--
   ---------------------------------------- 67.1/67.1 kB ? eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\pavan\anaconda3\python.exe -m pip install --upgrade pip setuptools wheel


In [9]:
!C:\Users\pavan\anaconda3\python.exe -m pip install --upgrade pip setuptools wheel
!pip install tensorflow opencv-python matplotlib

  Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 2.9/385.2 MB 18.7 MB/s eta 0:00:21
    --------------------------------------- 6.8/385.2 MB 19.1 MB/s eta 0:00:20
    --------------------------------------- 8.4/385.2 MB 17.9 MB/s eta 0:00:22
   - -------------------------------------- 15.5/385.2 MB 19.5 MB/s eta 0:00:19
   -- ------------------------------------- 22.0/385.2 MB 21.8 MB/s eta 0:00:17
   -- ------------------------------------- 28.0/385.2 MB 23.1 MB/s eta 0:00:16
   --

In [11]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [13]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [17]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
print("1")

1


In [29]:
# Setup paths
POS_PATH = os.path.join('data1', 'positive')
NEG_PATH = os.path.join('data1', 'negative')
ANC_PATH = os.path.join('data1', 'anchor')

In [31]:
# Make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [33]:
#2. Collect Positives and Anchors

In [35]:
#2.1 Untar Labelled Faces in the Wild Dataset

In [37]:
!tar -xf a.tgz

In [41]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

In [43]:
# 2.2 Collect Positive and Anchor Classes

In [45]:
# Import uuid library to generate unique image names
import uuid

In [47]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

'data1\\anchor\\285b2297-8f30-11ef-9eff-28c5d2259639.jpg'

In [60]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [ ]:
plt.imshow(frame[120:120+250,200:200+250, :])